In [ ]:
import boto3, re
from sagemaker import get_execution_role

Define the IAM role of execution

In [ ]:
role = get_execution_role()

Loads keras libraries

In [1]:
import keras
from keras.models import model_from_json


Using TensorFlow backend.


Creates directory with keras models

In [ ]:
!mkdir keras_model

In [ ]:
!ls keras_model
!pwd

In [2]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'sound_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/sound_model.h5')
print("Loaded model of feature extraction from disk")
loaded_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Loaded model of feature extraction from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 64, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 96, 64, 64)        640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 32, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 24, 16, 128)       0         
_________________________________________________________________
conv3/conv3_1 (Conv2D)       (None, 24, 16, 256)       295168    
_________________________

In [ ]:
!rm -rf export

# Export the Keras model (sound extractor) to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [ ]:
from keras import backend as K

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/'+ model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"vgg_features": loaded_model.output})

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [ ]:
!ls export/Servo/1

In [ ]:
!ls export/Servo/1/variables

In [ ]:
import tarfile
with tarfile.open('sound_extractor.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='sound_extractor.tar.gz', key_prefix='sound_extractor')

Creates an empty file

In [ ]:
!touch train.py

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/sound_extractor/sound_extractor.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [ ]:
import boto3
s3 = boto3.resource('s3')

my_bucket = s3.Bucket(sagemaker_session.default_bucket())

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.c5.large')

In [ ]:
!rm -rf export

# Does same thing for classifier model

In [ ]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'sound_class.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/sound_class.h5')
print("Loaded model of feature extraction from disk")
loaded_model.summary()

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from keras import backend as K

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/'+ model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"output": loaded_model.output})

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [ ]:
!ls export/Servo/1

In [ ]:
!ls export/Servo/1/variables

In [ ]:
import tarfile
with tarfile.open('sound_classifier.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='sound_classifier.tar.gz', key_prefix='sound_classifier')

In [ ]:
import boto3
s3 = boto3.resource('s3')

my_bucket = s3.Bucket(sagemaker_session.default_bucket())

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/sound_classifier/sound_classifier.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

## Download samples of acoustic scenes for testing

In [ ]:
!wget https://zenodo.org/record/1040168/files/TUT-acoustic-scenes-2017-evaluation.audio.1.zip

In [ ]:
!unzip TUT-acoustic-scenes-2017-evaluation.audio.1.zip

In [ ]:
!pip install --upgrade pip
!pip install resampy pysoundfile SoundFile

In [ ]:
!pwd

In [ ]:
import numpy as np
from scipy.io import wavfile
import six
import tensorflow as tf
import csv
import datetime
import numpy as np
import argparse
import csv
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/ml_osoai/audioset_sagemaker/audioset_feature_extractor')
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

In [ ]:
wav_file = './TUT-acoustic-scenes-2017-evaluation/audio/10.wav'
examples_batch = wavfile_to_examples(wav_file)

# Calculate pareto latence and cost of AWS service

In [ ]:
!pip install --upgrade pip
!pip install -U seaborn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

fig, ax = plt.subplots(figsize=(10,10))
fig.suptitle("Pareto optimum - Sound extractor")
df = pd.DataFrame({
'cost': [4.627,0.311,0.538,1.075,1.267,0.134,0.526,0.33,1.271],
'latence' : [496.53,791.06,560.83,501.68,536.8,498.23,543.46,518.53,536.46],
'group': ['ml.p3.2xlarge','ml.m4.xlarge','ml.c5.2xlarge','ml.c5.4xlarge','ml.c4.4xlarge','ml.c5.large','ml.c5.large\n ml.eia1.large','ml.c5.large\n ml.eia1.medium','ml.c5.4xlarge\n ml.eia1.medium']
})
#df['latence']=normalize(df['latence'].values)
#sns.set_palette(sns.color_palette("hls", 4))
sns.set_style("darkgrid")
paper_rc = {'lines.linewidth': 3}
sns.set_context("paper", rc = paper_rc)
p1=sns.scatterplot(x="cost", y="latence",data=df,marker="o",s=75)
p1.set_xlim(0.0, 5.0)
#p1.set_ylim(0.0,1.0)
p1.set_xlabel("Cost ($/h)")
p1.set_ylabel("Latence (ms)")
for line in range(0,df.shape[0]):
    p1.text(df.cost[line]+0.02, df.latence[line]-10, df.group[line], horizontalalignment='right', size='medium',rotation=0)
plt.show()

# Classifier based on Autopooling and BiLSTM

In [ ]:
!pip install autopool
!pip install keras-self-attention

In [ ]:
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/ml_osoai/audioset_sagemaker/audioset_feature_extractor')
import classifier_model
model = classifier_model.get_classifier_model(model_type='adaptative_pooling', model_path='/home/ec2-user/SageMaker/weights/md_50000_adap_iters.h5')

In [ ]:
!rm -rf export

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from keras import backend as K

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = '/home/ec2-user/SageMaker/export/Servo/'+ model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": model.input}, outputs={"output": model.output[0], "t_output":model.output[1]})

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()
import tarfile
with tarfile.open('sound_classifier_adapt.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='sound_classifier_adapt.tar.gz', key_prefix='sound_classifier_adapt')

Creates an empty file

In [ ]:
!touch train.py

In [ ]:
import boto3
s3 = boto3.resource('s3')

my_bucket = s3.Bucket(sagemaker_session.default_bucket())

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/sound_classifier_adapt/sound_classifier_adapt.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.c5.large')